# RDD Clave valor - Agregaciones

In [2]:
#Todo esto es la inicializacion, siempre necesitaremos uns parkcontext desde el cual empezaremos a trabajar
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()


# ReduceByKey()

In [6]:
#Creamos una lista con claves(key) repetidas
lista_tuplas = [('a',1),('b',2),('c',3),('d',4),('b',5),('a',6)]

In [7]:
#Creamos el pair RDD
pair_rdd = sc.parallelize(lista_tuplas)

In [8]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3), ('d', 4), ('b', 5), ('a', 6)]

In [5]:
#Como ya vimos en la sesion anterior podemos importar operadores como suma,resta,multiplicacion para usarlos en la agregacion
from operator import add,sub

In [10]:
pair_rdd.reduceByKey(add).collect()

[('a', 7), ('b', 7), ('c', 3), ('d', 4)]

In [11]:
#Como vemos, por cada clave, nos ha hecho la suma del valor, en el caso de a tenemos 6+1, 
#en el caso de b 5+2 c solo tiene 3 y d solo tiene 4

# FoldByKey()


In [17]:
#Vamos a hacer el mismo ejemplo con FoldByKey()
lista_tuplas = [('a',1),('b',2),('c',3),('d',4),('b',5),('a',6)]
pair_rdd = sc.parallelize(lista_tuplas,2)

In [18]:
pair_rdd.foldByKey(0,add).collect()

[('b', 7), ('c', 3), ('d', 4), ('a', 7)]

In [19]:
#En este caso igual que vimos en la sesion anterior tenemos que añadir el valor inicial por particiones, que en el caso de suma,
#Si ponemos un 0, el resultado va a ser el mismo que el de ReduceByKey()

In [20]:
pair_rdd.foldByKey(1,add).collect()

[('b', 9), ('c', 4), ('d', 5), ('a', 9)]

In [21]:
#Igual que vimos el fold si le añadimos un valor al ZeroValue, aplica en este caso la suma con ese valor, por particion
#entonces en este caso
pair_rdd.getNumPartitions()


2

In [ ]:
#vemos que el resultado en este caso varía, suma por cada key agruapda uno mas, 
#por ejemplo para el caso de a, que tenemos a,1 y a 6
# hará (1+1)+(6+1) = 9


# EJEMPLO DE LA DIAPOSITIVA



In [3]:

x = sc.parallelize([("a", 1), ("b", 1), ("a", 1), ("a", 1),
                    ("b", 1), ("b", 1), ("b", 1), ("b", 1)], 3)

In [7]:
#Con una funcion como en el ejemplo
def sumFunc(accum, n):
    return accum + n
 
y = x.reduceByKey(sumFunc)
y.collect()

[('b', 5), ('a', 3)]

In [8]:
#O con una funcion lambda:
y = x.reduceByKey(lambda accum, n: accum + n)
y.collect()

[('b', 5), ('a', 3)]

# Ejercicio 

In [3]:
compra = sc.parallelize([('pan',3),('agua',2),('azucar',1),('leche',2),('pan',1),('cereales',3),('agua',0.5),('leche',2),('filetes',5)])

In [19]:
compra.collect()

[('pan', 3),
 ('agua', 2),
 ('azucar', 1),
 ('leche', 2),
 ('pan', 1),
 ('cereales', 3),
 ('agua', 0.5),
 ('leche', 2),
 ('filetes', 5)]

In [6]:
#a) El total de cada producto
compra_a= compra.reduceByKey(add)

In [23]:
compra_a.collect()

[('agua', 2.5),
 ('pan', 4),
 ('azucar', 1),
 ('leche', 4),
 ('cereales', 3),
 ('filetes', 5)]

In [24]:
#b) Cuanto ha sido la vez que más caro le ha costado un producto

In [32]:
compra_b= compra.reduceByKey(lambda x,y: max(x,y))

In [33]:
compra_b.collect()

[('agua', 2),
 ('pan', 3),
 ('azucar', 1),
 ('leche', 2),
 ('cereales', 3),
 ('filetes', 5)]

# GroupByKey()

In [ ]:
Usando el ejemplo anterior, vamos a usar groupby en la lista de la compra ver que se ha gastado cada vez en cada producto

In [ ]:
compra = sc.parallelize([('pan',3),('agua',2),('azucar',1),('leche',2),('pan',1),('cereales',3),('agua',0.5),('leche',2),('filetes',5)])
compra.collect()

In [64]:
compra_groupbykey = compra.groupByKey()

In [65]:
compra_groupbykey.collect()

[('agua', <pyspark.resultiterable.ResultIterable at 0x188d04f3d60>),
 ('pan', <pyspark.resultiterable.ResultIterable at 0x188d04f32e0>),
 ('azucar', <pyspark.resultiterable.ResultIterable at 0x188d04f3880>),
 ('leche', <pyspark.resultiterable.ResultIterable at 0x188d04f3430>),
 ('cereales', <pyspark.resultiterable.ResultIterable at 0x188d04f3250>),
 ('filetes', <pyspark.resultiterable.ResultIterable at 0x188d04f37f0>)]

In [66]:
lista_compra = [(x,list(y)) for x,y in compra_groupbykey.collect()]

In [67]:
lista_compra

[('agua', [2, 0.5]),
 ('pan', [3, 1]),
 ('azucar', [1]),
 ('leche', [2, 2]),
 ('cereales', [3]),
 ('filetes', [5])]

# CombineByKey(createCombiner(),mergeValue(),mergeCombiners())

In [22]:
lista = sc.parallelize([('a',1),('b',3),('a',2),('b',1),('c',4),('a',1)],2)
lista.glom().collect() #Con glom vemos las particiones

[[('a', 1), ('b', 3), ('a', 2)], [('b', 1), ('c', 4), ('a', 1)]]

In [27]:
#Lo que hace es, el primer valor, solo le decimos que coja la clave, el segundo que por cada particion sume los valores, y
#el tercero que de cada particion, reste

#En la primera particion tendremos a:3 b:3 
#en la segundoa particion tenemos a:1 b:1 c:4
#Ahora restamos particiones: a:2 b:3 c:4

lista.combineByKey(lambda x:x,lambda x,y:x+y,lambda x,y:x-y).collect()

[('b', 2), ('c', 4), ('a', 2)]

In [30]:
#Por ejempl podemos sumar por particion y luego quedanos con la maxima:
lista.combineByKey(lambda x:x,lambda x,y:x+y,lambda x,y:max(x,y)).collect()

[('b', 3), ('c', 4), ('a', 3)]

In [82]:
compra = sc.parallelize([('pan',3),('agua',2),('azucar',1),('leche',2),('pan',1),('cereales',3),('agua',0.5),('leche',2),('filetes',5)],2)


In [84]:
compra.glom().collect()

[[('pan', 3), ('agua', 2), ('azucar', 1), ('leche', 2)],
 [('pan', 1), ('cereales', 3), ('agua', 0.5), ('leche', 2), ('filetes', 5)]]

In [91]:




lista_importe_veces = compra.combineByKey(lambda x:(x),#Clave
                                          lambda x,y:x+y,#Acumulador por cada una de las particiones
                                          lambda x,y:x+y) #Acumulador por particiones 



In [92]:
lista_importe_veces.collect()

[('leche', 4),
 ('cereales', 3),
 ('filetes', 5),
 ('pan', 4),
 ('agua', 2.5),
 ('azucar', 1)]

In [93]:
#Para verlo mejor vamos a añadir a la priemra particion otra compra de pan
compra = sc.parallelize([('pan',3),('pan',5),('agua',2),('azucar',1),('leche',2),('pan',1),('cereales',3),('agua',0.5),('leche',2),('filetes',5)],2)


In [94]:
compra.glom().collect()

[[('pan', 3), ('pan', 5), ('agua', 2), ('azucar', 1), ('leche', 2)],
 [('pan', 1), ('cereales', 3), ('agua', 0.5), ('leche', 2), ('filetes', 5)]]

In [96]:
#Ahora tenemos en la primera particion dos de pan, elegiremos la clave el producto, por cada paticion vamos a sumar,
#y agruparemos las particiones aplicando una resta entonces
#En el caso del pan, primero sumara:
#Particion 1: pan 8
#Particion 2: Pan 1
#Resultado final: Pan 7

In [97]:
lista_importe_veces = compra.combineByKey(lambda x:(x),#Clave
                                          lambda x,y:x+y,#Acumulador por particion
                                          lambda x,y:x-y) #Acumulador por particiones

In [101]:
lista_importe_veces.collect()

[('leche', 0),
 ('cereales', 3),
 ('filetes', 5),
 ('pan', 7),
 ('agua', 1.5),
 ('azucar', 1)]